## Libraries

In [43]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MaxAbsScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import check_random_state
from sklearn.datasets import load_boston
import pandas as pd
from pathlib import Path

In [44]:
from bayes_opt import BayesianOptimization
from bayes_opt import SequentialDomainReductionTransformer
from bayes_opt import UtilityFunction
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events

In [45]:
%run SVR_LS_Library.ipynb 

## Load data

In [46]:
rng = check_random_state(0)
boston = load_boston()
perm = rng.permutation(boston.target.size)
boston.data = boston.data[perm]
boston.target = boston.target[perm]
# 
Y = boston.target
X = boston.data

In [47]:
# X = pd.read_csv("new_boston.csv")

In [48]:
#split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state = 5)

#normalize
scaler = MaxAbsScaler().fit(x_train)
X_train = scaler.transform(x_train); X_test = scaler.transform(x_test)

scaler1 = MaxAbsScaler().fit(y_train.reshape(-1, 1))
Y_train = scaler1.transform(y_train.reshape(-1, 1)).reshape(-1)
Y_test = scaler1.transform(y_test.reshape(-1, 1)).reshape(-1)

## Optimization function

In [49]:
def opt_bas(C, gamma):
    
    # parameters
    hyperparameters = {
        'kernel' : "rbf",
        'C' : C, 
        'gamma' : gamma,
    }
    
    # fit and predict
    model = SVR_ls(**hyperparameters).fit(X_train, Y_train)

    y_pred = model.predict(X_test)
    
    # rescale
#     y_pred = scaler1.inverse_transform(predict.reshape(-1, 1)).reshape(-1)
    
    # get score
    mape = np.mean(np.abs((Y_test - y_pred)/Y_test))*100
    
    return -mape

## Run optimization

In [50]:
class newJSONLogger(JSONLogger):

      def __init__(self, path):
            self._path=None
            super(JSONLogger, self).__init__()
            self._path = path if path[-5:] == ".json" else path + ".json"

In [143]:
# Bounded region of parameter space
pbounds = {'C': (0.001, 1000000), 'gamma': (0.0001, 10)}

# Domain reduction function
# bounds_transformer = SequentialDomainReductionTransformer()

# Bayes optimizer instantiation
optimizer = BayesianOptimization(f=opt_bas, 
                                 pbounds=pbounds, 
                                 random_state=1, verbose=2, 
#                                  bounds_transformer=bounds_transformer
                                )

# keep data
log_path = Path().resolve() / "Logs" / "test1.json"
logger = newJSONLogger(path = str(log_path))
optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)

In [144]:
optimizer.probe(params={"C": 131.6135372548593, "gamma": 0.1926931038470609}, lazy = False)
optimizer.probe(params={"C": 4, "gamma": 2}, lazy = False)
optimizer.probe(params={"C": 7.628874476243877, "gamma": 1.1659271563361087}, lazy = False)

In [ ]:
optimizer.maximize(init_points=5, n_iter=400)

{"target": -10.405396953891422, "params": {"C": 145.77384906735986, "gamma": 0.2124613849674528}, "datetime": {"datetime": "2021-06-18 11:04:40", "elapsed": 6209.407793, "delta": 11.441449}}

{"target": -10.514071087943606, "params": {"C": 146.55652561348376, "gamma": 0.24100561527179876}, "datetime": {"datetime": "2021-06-18 10:39:32", "elapsed": 4702.323429, "delta": 13.800381}}

{"target": -10.917141024687126, "params": {"C": 144.95313389424803, "gamma": 0.34218534204829437}, "datetime": {"datetime": "2021-06-18 10:04:16", "elapsed": 2586.015739, "delta": 7.89459}}

{"target": -11.340590596189683, "params": {"C": 135.11776824073686, "gamma": 0.4207548977595412}, "datetime": {"datetime": "2021-06-18 10:00:56", "elapsed": 2385.807744, "delta": 5.368786}}

{"target": -10.273963393355416, "params": {"C": 131.6135372548593, "gamma": 0.1926931038470609}, "datetime": {"datetime": "2021-06-18 09:56:55", "elapsed": 2145.094956, "delta": 6.295304}}

{"target": -10.515324650130054, "params": {"C": 133.41030297171602, "gamma": 0.25634656923630994}, "datetime": {"datetime": "2021-06-18 09:43:23", "elapsed": 1333.236594, "delta": 3.800363}}

{"target": -10.509953420444594, "params": {"C": 90.56546154667917, "gamma": 0.3265541455292291}, "datetime": {"datetime": "2021-06-18 09:36:11", "elapsed": 900.641656, "delta": 4.047269}}

{"target": -10.482047036160049, "params": {"C": 106.37884794241084, "gamma": 0.2865778921210128}, "datetime": {"datetime": "2021-06-18 09:30:28", "elapsed": 557.583728, "delta": 2.382331}}

{"target": -10.189788195327274, "params": {"C": 106.34314342077398, "gamma": 0.19081778184382325}, "datetime": {"datetime": "2021-06-18 09:25:14", "elapsed": 243.842363, "delta": 1.564244}}

{"target": -9.288636246050661, "params": {"C": 7.115276433808854, "gamma": 1.2427971926254497}, "datetime": {"datetime": "2021-06-17 22:00:42", "elapsed": 17.409482, "delta": 0.274522}}

{"target": -9.390351622404006, "params": {"C": 5.56244671588079, "gamma": 1.364638611084284}, "datetime": {"datetime": "2021-06-17 22:00:46", "elapsed": 20.634438, "delta": 0.291262}}

{"target": -9.273151468081213, "params": {"C": 7.429108586210357, "gamma": 1.1682991477100433}, "datetime": {"datetime": "2021-06-17 22:00:57", "elapsed": 32.137852, "delta": 0.900288}}

{"target": -9.398309483014152, "params": {"C": 7.263608156940218, "gamma": 0.9419059306485307}, "datetime": {"datetime": "2021-06-17 22:00:55", "elapsed": 29.812497, "delta": 0.961556}}

{"target": -9.269896691232166, "params": {"C": 7.433926922431135, "gamma": 1.1756996388768455}, "datetime": {"datetime": "2021-06-17 22:01:07", "elapsed": 42.054071, "delta": 0.86324}}

{"target": -9.348638522020378, "params": {"C": 9.703492422040622, "gamma": 0.8721642844446199}, "datetime": {"datetime": "2021-06-17 22:01:44", "elapsed": 79.309359, "delta": 1.381108}}

{"target": -9.286146249516818, "params": {"C": 8.41359372018643, "gamma": 1.070185910497249}, "datetime": {"datetime": "2021-06-17 22:03:09", "elapsed": 164.505163, "delta": 1.244778}}

{"target": -9.297613526377074, "params": {"C": 9.784500524430815, "gamma": 1.155192591018298}, "datetime": {"datetime": "2021-06-17 22:04:05", "elapsed": 219.5999, "delta": 1.163145}}

{"target": -9.27036097690317, "params": {"C": 9.030072196367662, "gamma": 1.0975712875360908}, "datetime": {"datetime": "2021-06-17 22:04:20", "elapsed": 234.94536, "delta": 1.210502}}

{"target": -9.269694966254036, "params": {"C": 9.470540429368913, "gamma": 1.038282143064161}, "datetime": {"datetime": "2021-06-17 22:04:32", "elapsed": 246.894324, "delta": 1.186165}}

{"target": -9.339703800205426, "params": {"C": 8.05208254958713, "gamma": 0.99417278582112}, "datetime": {"datetime": "2021-06-17 22:04:53", "elapsed": 268.057744, "delta": 1.40043}}

{"target": -9.275239984640043, "params": {"C": 8.597610101717859, "gamma": 1.1643192103084814}, "datetime": {"datetime": "2021-06-17 22:08:19", "elapsed": 473.867519, "delta": 1.671379}}

{"target": -9.271419721641024, "params": {"C": 7.96671064923825, "gamma": 1.2037037725528934}, "datetime": {"datetime": "2021-06-17 22:08:41", "elapsed": 495.740782, "delta": 2.890053}}

{"target": -9.26732766401765, "params": {"C": 7.628874476243877, "gamma": 1.1659271563361087}, "datetime": {"datetime": "2021-06-17 22:10:41", "elapsed": 616.10778, "delta": 2.225994}}

In [ ]:
# from bayes_opt.util import load_logs


# new_optimizer = BayesianOptimization(
#     f=black_box_function,
#     pbounds={"x": (-2, 2), "y": (-2, 2)},
#     verbose=2,
#     random_state=7,
# )
# log_path = Path().resolve() / "Logs" / "test2.json"
# # New optimizer is loaded with previously seen points
# load_logs(new_optimizer, logs=["./logs.json"]);